# 👒 [E-14] Movielens 영화 추천 실습

이전에 다룬 MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템 제작해보기.

### 🐬 Movielens 데이터
: 추천 시스템의 MNIST라 부를만한 Movielens 데이터

<br/>

**✔️ 조건**
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있음: **`MovieLens 1M Dataset`** 사용 권장
- 별점 데이터는 대표적인 explicit 데이터 -> implicit 데이터로 간주하고 테스트해보기
- 별점을 **시청횟수**로 해석하기
- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하기

### 1) 데이터 준비와 전처리
Movielens 데이터를 보면 `rating.dat` 안에 이미 인덱싱까지 완료한 사용자-영화-평점 데이터가 정리되어 있음

In [1]:
import pandas as pd
import os

In [2]:
rating_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'

ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding="ISO-8859-1")

orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings'] >= 3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'

cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### - 추가적인 데이터 탐색

In [8]:
movies.tail() # index와 id가 다르구나.

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [9]:
sum = 0
for i in range(len(movies)):
    if (i+1) != movies['movie_id'][i]:
#         print(i)
        sum += 1
    
print(sum)
# movies['movie_id']

3793


In [10]:
movies[88:93] # movie_id가 1부터 차례대로 존재하지 않는다.

,movie_id,title,genre
88,89,Nick of Time (1995),Action|Thriller
89,90,"Journey of August King, The (1995)",Drama
90,92,Mary Reilly (1996),Drama|Thriller
91,93,Vampire in Brooklyn (1995),Comedy|Romance
92,94,Beautiful Girls (1996),Drama


In [91]:
ratings['user_id'].head()

0    1
1    1
2    1
3    1
4    1
Name: user_id, dtype: int64

In [90]:
ratings['user_id'].tail()

1000203    6040
1000205    6040
1000206    6040
1000207    6040
1000208    6040
Name: user_id, dtype: int64

In [89]:
ratings['user_id'].nunique()

# user_id도 인덱스는 1~6040이라 총 6040이라 생각했지만,
# nunique()로 판별해보니 총 6039개였다.
# 마찬가지로 중간에 빠진 id가 있다. (1부터 차례대로 존재하지 않는다.)

6039

- movie_id를 기준으로 합쳐 새로운 데이터 프레임 만들기
> - `how=left` 속성을 통해 ratings 데이터에 movies를 붙이고, `on='movie_id'` 속성을 통해 `movie_id`를 기준으로 합친다.

In [12]:
len(ratings), len(movies)

(836478, 3883)

In [13]:
df = pd.merge(ratings, movies, how='left', on='movie_id')
df.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [14]:
df.groupby('user_id')['movie_id'].count().describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [15]:
df['movie_id'].nunique(), df['title'].nunique()

(3628, 3628)

### 2) 분석해보기

> ratings에 있는 유니크한 영화 개수

In [16]:
ratings['movie_id'].nunique()

3628

In [17]:
df['movie_id'].nunique()

3628

> ratings에 있는 유니크한 사용자 수

In [18]:
ratings['user_id'].nunique()

6039

In [19]:
df['user_id'].nunique()

6039

> 가장 인기 있는 영화 30개 (인기순)

In [20]:
df.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [21]:
df.groupby('title').count()['counts'].sort_values(ascending=False)[:30]

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 3) 내가 선호하는 영화를 5가지 골라 ratings에 추가해주기
> 일단 검색이 easy하기 위해 title을 소문자로 바꿔준다.

In [22]:
df['title'] = df['title'].str.lower()

In [23]:
df.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,978302109,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,my fair lady (1964),Musical|Romance
3,1,3408,4,978300275,erin brockovich (2000),Drama
4,1,2355,5,978824291,"bug's life, a (1998)",Animation|Children's|Comedy


> 필요한 열만 냅두기
>> - `movie_id`와 `title`은 실질적으로 같은 의미라 판단 -> `movie_id` 제거
>> - `timestamp` 제거

In [24]:
data_copy = df.copy()

In [25]:
data = data_copy.drop(['movie_id', 'timestamp'], axis=1)
data.head()

,user_id,counts,title,genre
0,1,5,one flew over the cuckoo's nest (1975),Drama
1,1,3,james and the giant peach (1996),Animation|Children's|Musical
2,1,3,my fair lady (1964),Musical|Romance
3,1,4,erin brockovich (2000),Drama
4,1,5,"bug's life, a (1998)",Animation|Children's|Comedy


> - misérables, les (1998)
> - king kong (1933)
> - men in black (1997)
> - soldier's story, a (1984)
> - back to the future (1985)

In [26]:
print(
data[data['title'] == "misérables, les (1998)"][:1]['genre'],
data[data['title'] == "king kong (1933)"][:1]['genre'],
data[data['title'] == "men in black (1997)"][:1]['genre'],
data[data['title'] == "soldier's story, a (1984)"][:1]['genre'],
data[data['title'] == "back to the future (1985)"][:1]['genre'])

99    Drama
Name: genre, dtype: object 228    Action|Adventure|Horror
Name: genre, dtype: object 185    Action|Adventure|Comedy|Sci-Fi
Name: genre, dtype: object 96    Drama
Name: genre, dtype: object 22    Comedy|Sci-Fi
Name: genre, dtype: object


In [27]:
my_fav = ['misérables, les (1998)', 'king kong (1933)', 'men in black (1997)', "soldier's story, a (1984)", 'back to the future (1985)']
gen = ['Drama', 'Action|Adventure|Horror', 'Action|Adventure|Comedy|Sci-Fi', 'Drama', 'Comedy|Sci-Fi']

my_movie = pd.DataFrame({'user_id': ['yeon']*5,
                         'counts':[5]*5,
                         'title': my_fav,
                         'genre': gen
                         })

if not data.isin({'user_id': ['yeon']})['user_id'].any():  # user_id에 'yeon'이라는 데이터가 없다면
    data = data.append(my_movie)                           # 위에 임의로 만든 my_movie 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,counts,title,genre
836473,6040,3,platoon (1986),Drama|War
836474,6040,5,"crying game, the (1992)",Drama|Romance|War
836475,6040,5,welcome to the dollhouse (1995),Comedy|Drama
836476,6040,4,sophie's choice (1982),Drama
836477,6040,4,e.t. the extra-terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,yeon,5,"misérables, les (1998)",Drama
1,yeon,5,king kong (1933),Action|Adventure|Horror
2,yeon,5,men in black (1997),Action|Adventure|Comedy|Sci-Fi
3,yeon,5,"soldier's story, a (1984)",Drama
4,yeon,5,back to the future (1985),Comedy|Sci-Fi


In [28]:
data['user_id'].nunique(), data['title'].nunique(), data['genre'].nunique()

(6040, 3628, 301)

In [29]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v: k for k, v in enumerate(user_unique)} # id: index
title_to_idx = {v: k for k, v in enumerate(title_unique)} # artist: index

In [30]:
len(user_unique), len(title_unique)

(6040, 3628)

In [31]:
print(user_to_idx['yeon']) # yeon이 마지막 유저 인덱스로 잘 들어왔다.
print(title_to_idx["misérables, les (1998)"])

6039
98


In [32]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title,genre
0,0,5,0,Drama
1,0,3,1,Animation|Children's|Musical
2,0,3,2,Musical|Romance
3,0,4,3,Drama
4,0,5,4,Animation|Children's|Comedy
...,...,...,...,...
0,6039,5,98,Drama
1,6039,5,196,Action|Adventure|Horror
2,6039,5,175,Action|Adventure|Comedy|Sci-Fi
3,6039,5,95,Drama


### 4) CSR Matrix 직접 만들기

In [33]:
data['user_id'].nunique(), data['title'].nunique()

(6040, 3628)

In [34]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape=(num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [35]:
print(csr_data)

  (0, 0)	5
  (0, 1)	3
  (0, 2)	3
  (0, 3)	4
  (0, 4)	5
  (0, 5)	3
  (0, 6)	5
  (0, 7)	5
  (0, 8)	4
  (0, 9)	4
  (0, 10)	5
  (0, 11)	4
  (0, 12)	4
  (0, 13)	4
  (0, 14)	5
  (0, 15)	4
  (0, 16)	3
  (0, 17)	4
  (0, 18)	5
  (0, 19)	4
  (0, 20)	3
  (0, 21)	3
  (0, 22)	5
  (0, 23)	5
  (0, 24)	3
  :	:
  (6038, 2311)	4
  (6038, 2317)	5
  (6038, 2386)	4
  (6038, 2394)	5
  (6038, 2424)	4
  (6038, 2437)	4
  (6038, 2446)	5
  (6038, 2471)	4
  (6038, 2511)	5
  (6038, 2523)	4
  (6038, 2559)	3
  (6038, 2560)	4
  (6038, 2631)	5
  (6038, 2648)	4
  (6038, 2654)	5
  (6038, 2738)	4
  (6038, 2740)	5
  (6038, 2857)	5
  (6038, 2860)	3
  (6038, 3311)	5
  (6039, 22)	5
  (6039, 95)	5
  (6039, 98)	5
  (6039, 175)	5
  (6039, 196)	5


### 5) als_model = AlternatingLeastSquares 모델을 직접 구성해 훈련 시키기

In [36]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['MKL_NUM_THREADS'] = '1'

In [37]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=15, 
                                    dtype=np.float32)

In [38]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [39]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도 파악해보기

In [40]:
yeon, miserable = user_to_idx['yeon'], title_to_idx['misérables, les (1998)']
yeon_vector, miserable_vector = als_model.user_factors[yeon], als_model.item_factors[miserable]

In [41]:
yeon_vector

array([ 0.10223146,  0.41567594, -0.27738497,  0.12017343, -0.41949466,
       -0.04357485, -0.01972907,  0.4091624 ,  0.4892804 ,  0.54615086,
        0.11377988,  0.2117782 ,  0.0088392 ,  0.50712824, -0.4280614 ,
        0.22757185,  0.61680824, -0.10853993, -0.09536171, -0.32253507,
       -0.1808744 ,  0.14798819,  1.0483569 ,  1.1636428 ,  0.16501772,
       -0.13266475, -0.6215812 , -0.02033659,  0.2740878 ,  0.16976358,
        0.30697903,  0.28227216,  0.57928264, -0.31713188,  0.19659536,
       -0.11899117,  0.81394875, -0.19006501, -0.7696614 ,  0.12688126,
       -0.3400262 , -0.5874306 , -0.55463994,  0.6241994 ,  0.60745037,
       -0.6577938 , -0.6739609 , -0.37751538,  0.00982179,  0.7640779 ,
       -0.6070048 , -0.37419865,  0.3471502 , -0.3369955 , -0.29753348,
       -0.5776674 , -0.26770255,  0.03486999,  0.14616643, -0.09931198,
        0.69273734,  0.93203104,  0.02886639,  0.531077  ,  0.3857499 ,
        0.17034037, -0.3820886 , -0.21078633,  0.20665598, -0.01

In [42]:
miserable_vector

array([-0.00388683,  0.03442129, -0.01294139, -0.02207728, -0.01233814,
        0.0045761 ,  0.00944578,  0.01094749,  0.02332033,  0.024163  ,
        0.01982096, -0.00696979,  0.02119031,  0.00399983,  0.00405479,
        0.02025165,  0.01207358,  0.00851654,  0.01801369,  0.01934545,
       -0.01299634,  0.01437124,  0.00571197,  0.01322639,  0.01525709,
       -0.01032946,  0.00572218,  0.01056943,  0.01485178,  0.02471231,
       -0.01584918,  0.01956293,  0.00331143,  0.01119871,  0.00033239,
       -0.00238855, -0.0019167 ,  0.00140023, -0.0085923 ,  0.01794661,
        0.02230695, -0.01170717, -0.00353177, -0.00477717,  0.02181477,
        0.00565632,  0.00485325, -0.0186134 ,  0.01229157,  0.02689637,
        0.01556346, -0.0069597 ,  0.01195382, -0.01258907,  0.00667704,
        0.01521775, -0.00065525,  0.01368804,  0.00274175,  0.00563149,
       -0.0098778 ,  0.01916235,  0.01845099,  0.00990553, -0.00039155,
       -0.01311252, -0.00845211, -0.0011868 ,  0.00481899,  0.00

In [43]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(yeon_vector, miserable_vector)

0.18980479

In [44]:
# print(
# data[data['title'] == "misérables, les (1998)"][:1]['genre'],
# data[data['title'] == "king kong (1933)"][:1]['genre'],
# data[data['title'] == "men in black (1997)"][:1]['genre'],
# data[data['title'] == "soldier's story, a (1984)"][:1]['genre'],
# data[data['title'] == "back to the future (1985)"][:1]['genre'])

In [45]:
king = title_to_idx["king kong (1933)"]
king_vector = als_model.item_factors[king]

men = title_to_idx["men in black (1997)"]
men_vector = als_model.item_factors[men]

soldier = title_to_idx["soldier's story, a (1984)"]
soldier_vector = als_model.item_factors[soldier]

future = title_to_idx["back to the future (1985)"]
future_vector = als_model.item_factors[future]

In [46]:
print(np.dot(yeon_vector, king_vector),
np.dot(yeon_vector, men_vector), 
np.dot(yeon_vector, soldier_vector), 
np.dot(yeon_vector, future_vector))

0.27823105 0.48559764 0.09167732 0.40583813


> 생각보다 선호도가 1과 차이가 많이 나고, 심지어는 0 근처의 값들도 많이 나왔다.

#### 보지 못한 영화에 대한 선호도 확인

In [47]:
platoon = title_to_idx["platoon (1986)"]
platoon_vector = als_model.item_factors[platoon]

np.dot(yeon_vector, platoon_vector)

-0.1247444

In [48]:
sophie = title_to_idx["sophie's choice (1982)"]
sophie_vector = als_model.item_factors[sophie]

np.dot(yeon_vector, sophie_vector)

-0.022222796

In [49]:
doll = title_to_idx["welcome to the dollhouse (1995)"]
doll_vector = als_model.item_factors[doll]

np.dot(yeon_vector, doll_vector)

0.062141195

> 보지 못한 영화에 대한 선호도가 모두 음수가 나올 줄 알았는데 0.06인 값도 나왔다. 이는 yeon이 시청한 영화 `soldier's story a`와 0.02밖에 차이가 나지 않는다.

### 8) 내가 가장 좋아할만한 영화들 추천받기
### 8-1) 비슷한 영화 찾기

In [50]:
favorite_title = "men in black (1997)"
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(175, 1.0),
 (107, 0.831708),
 (92, 0.61478126),
 (62, 0.57132185),
 (150, 0.49441674),
 (145, 0.48883635),
 (82, 0.48881575),
 (124, 0.48745483),
 (375, 0.45425677),
 (138, 0.40729585),
 (3466, 0.40442565),
 (60, 0.35737863),
 (3183, 0.35596535),
 (670, 0.35376173),
 (544, 0.35368767)]

In [51]:
# title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v: k for k, v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'independence day (id4) (1996)',
 'fifth element, the (1997)',
 'lost world: jurassic park, the (1997)',
 'matrix, the (1999)',
 'face/off (1997)',
 'true lies (1994)',
 'schlafes bruder (brother of sleep) (1995)',
 'star wars: episode i - the phantom menace (1999)',
 'sorority house massacre ii (1990)',
 'galaxy quest (1999)',
 'stargate (1994)']

In [52]:
# 위를 함수로 구현하기
def get_similar_title(title_name: str): # 타입 힌트
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

get_similar_title("men in black (1997)")

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'independence day (id4) (1996)',
 'fifth element, the (1997)',
 'lost world: jurassic park, the (1997)',
 'matrix, the (1999)',
 'face/off (1997)',
 'true lies (1994)']

### 8-2) 내가 좋아할 만한 영화 추천

In [53]:
user = user_to_idx['yeon']

# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, 
                                        csr_data, 
                                        N=20, 
                                        filter_already_liked_items=True)
title_recommended

[(107, 0.44837394),
 (92, 0.28796017),
 (670, 0.26986974),
 (0, 0.25429463),
 (44, 0.23039778),
 (124, 0.2219212),
 (243, 0.21735065),
 (60, 0.21349567),
 (64, 0.20621061),
 (424, 0.19960901),
 (62, 0.19337395),
 (87, 0.18880908),
 (117, 0.18792754),
 (450, 0.18114169),
 (6, 0.17762613),
 (82, 0.16795927),
 (5, 0.16780601),
 (1004, 0.16420367),
 (154, 0.15631124),
 (26, 0.14648208)]

In [54]:
[idx_to_title[i[0]] for i in title_recommended]

['jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'galaxy quest (1999)',
 "one flew over the cuckoo's nest (1975)",
 'star wars: episode iv - a new hope (1977)',
 'matrix, the (1999)',
 'ghostbusters (1984)',
 'star wars: episode i - the phantom menace (1999)',
 'star wars: episode vi - return of the jedi (1983)',
 'misérables, les (1995)',
 'total recall (1990)',
 'braveheart (1995)',
 'star wars: episode v - the empire strikes back (1980)',
 'life is beautiful (la vita è bella) (1997)',
 'ben-hur (1959)',
 'lost world: jurassic park, the (1997)',
 'princess bride, the (1987)',
 'frankenstein (1931)',
 'as good as it gets (1997)',
 'e.t. the extra-terrestrial (1982)']

#### 기여도 확인

In [55]:
jurassic = title_to_idx['jurassic park (1993)']
explain = als_model.explain(user, csr_data, itemid=jurassic)

In [56]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('men in black (1997)', 0.347568738931146),
 ('king kong (1933)', 0.033649843947351585),
 ('misérables, les (1998)', 0.030933253977570685),
 ('back to the future (1985)', 0.028904132256918073),
 ("soldier's story, a (1984)", 0.0009355589975818424)]

> 내가 봤다고 추가한 영화들이 `jurassic park` 영화를 추천하는데 가장 많이 기여했음을 알 수 있다.

## 🐬 2nd-try
> 내가 좋아하는 영화를 좀 더 의미있게 추출해보자.
> - 비슷한 장르만 봤다고 가정하기

In [57]:
data_copy2 = df.copy()

In [58]:
data = data_copy2.drop(['movie_id', 'timestamp'], axis=1)
data.head()

,user_id,counts,title,genre
0,1,5,one flew over the cuckoo's nest (1975),Drama
1,1,3,james and the giant peach (1996),Animation|Children's|Musical
2,1,3,my fair lady (1964),Musical|Romance
3,1,4,erin brockovich (2000),Drama
4,1,5,"bug's life, a (1998)",Animation|Children's|Comedy


In [59]:
# 'Animation'이 포함된 장르만 시청했다고 가정
data[data['genre'].str.contains('Animation')][100:120]

,user_id,counts,title,genre
1734,18,3,all dogs go to heaven (1989),Animation|Children's
1744,18,5,mulan (1998),Animation|Children's
1745,18,5,aladdin (1992),Animation|Children's|Comedy|Musical
1748,18,4,toy story (1995),Animation|Children's|Comedy
1752,18,5,charlotte's web (1973),Animation|Children's
1754,18,5,"secret of nimh, the (1982)",Animation|Children's
1761,18,5,snow white and the seven dwarfs (1937),Animation|Children's|Musical
1762,18,5,beauty and the beast (1991),Animation|Children's|Musical
1763,18,4,pinocchio (1940),Animation|Children's
1772,18,5,"american tail, an (1986)",Animation|Children's|Comedy


In [60]:
my_fav = ["tarzan (1999)", "toy story (1995)", "cinderella (1950)",
         "beauty and the beast (1991)", "bambi (1942)"]

my_movie = pd.DataFrame({'user_id': ['yeon']*5,
                         'counts': [4]*5,
                         'title': my_fav,
                         'genre': ['Animation']*5
                        })

if not data.isin({'user_id': ['yeon']})['user_id'].any():
    data = data.append(my_movie)
    
data.tail(10)

,user_id,counts,title,genre
836473,6040,3,platoon (1986),Drama|War
836474,6040,5,"crying game, the (1992)",Drama|Romance|War
836475,6040,5,welcome to the dollhouse (1995),Comedy|Drama
836476,6040,4,sophie's choice (1982),Drama
836477,6040,4,e.t. the extra-terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,yeon,4,tarzan (1999),Animation
1,yeon,4,toy story (1995),Animation
2,yeon,4,cinderella (1950),Animation
3,yeon,4,beauty and the beast (1991),Animation
4,yeon,4,bambi (1942),Animation


In [61]:
data.isin({'user_id': ['yeon']}).any()

user_id     True
counts     False
title      False
genre      False
dtype: bool

In [62]:
data.isin({'user_id': ['yeon']})['user_id'].any()

True

In [63]:
data['user_id'].nunique(), data['title'].nunique(), data['genre'].nunique()

(6040, 3628, 301)

In [64]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀을 indexing하는 코드
user_to_idx = {v: k for k, v in enumerate(user_unique)}
title_to_idx = {v: k for k, v in enumerate(title_unique)}

In [65]:
print(user_to_idx['yeon']) # 마지막 index로 잘 들어옴

6039


In [66]:
user_to_idx.get

<function dict.get(key, default=None, /)>

In [67]:
# user_to_idx.get을 통해 user_to_idx의 모든 key, value를 Series로 구함
# .map()을 통해 data['user_id']와 같은 key를 갖는 value를 map 시켜줌
# 정상적으로 인덱싱 되지 않은 row의 경우 인ㄷ게스가 NaN이 될테니 dropna()로 제거
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data # data['user_id']로 인덱싱된 Series로 교체해줌
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title,genre
0,0,5,0,Drama
1,0,3,1,Animation|Children's|Musical
2,0,3,2,Musical|Romance
3,0,4,3,Drama
4,0,5,4,Animation|Children's|Comedy
...,...,...,...,...
0,6039,4,16,Animation
1,6039,4,40,Animation
2,6039,4,37,Animation
3,6039,4,10,Animation


In [68]:
# CSR Matrix
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape=(num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [69]:
print(csr_data)

  (0, 0)	5
  (0, 1)	3
  (0, 2)	3
  (0, 3)	4
  (0, 4)	5
  (0, 5)	3
  (0, 6)	5
  (0, 7)	5
  (0, 8)	4
  (0, 9)	4
  (0, 10)	5
  (0, 11)	4
  (0, 12)	4
  (0, 13)	4
  (0, 14)	5
  (0, 15)	4
  (0, 16)	3
  (0, 17)	4
  (0, 18)	5
  (0, 19)	4
  (0, 20)	3
  (0, 21)	3
  (0, 22)	5
  (0, 23)	5
  (0, 24)	3
  :	:
  (6038, 2311)	4
  (6038, 2317)	5
  (6038, 2386)	4
  (6038, 2394)	5
  (6038, 2424)	4
  (6038, 2437)	4
  (6038, 2446)	5
  (6038, 2471)	4
  (6038, 2511)	5
  (6038, 2523)	4
  (6038, 2559)	3
  (6038, 2560)	4
  (6038, 2631)	5
  (6038, 2648)	4
  (6038, 2654)	5
  (6038, 2738)	4
  (6038, 2740)	5
  (6038, 2857)	5
  (6038, 2860)	3
  (6038, 3311)	5
  (6039, 10)	4
  (6039, 16)	4
  (6039, 17)	4
  (6039, 37)	4
  (6039, 40)	4


In [70]:
# AlternatingLeastSquares 모델

from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하는 부분 (학습 내용과는 무관)
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['MKL_NUM_THREADS'] = '1'

In [71]:
# Implicit AlternatingSeriesSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100,
                                    regularization=0.01,
                                    use_gpu=False,
                                    iterations=15,
                                    dtype=np.float32)

In [72]:
# als 모델은 input으로 item X user 꼴의 matrix를 받으므로 Transpose 해주기
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [73]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [74]:
# 내가 선호하는 영화 하나 골라 훈련된 모델이 예측한 내 선호도 파악해보기
yeon, toy = user_to_idx['yeon'], title_to_idx['toy story (1995)']
yeon_vector, toy_vector = als_model.user_factors[yeon], als_model.item_factors[toy]

In [75]:
yeon_vector

array([-0.03237853, -0.8112134 ,  0.12492882, -0.03238954,  0.16518976,
       -0.13444537,  0.46394488, -0.45214972,  1.0664    ,  0.14171718,
        0.32324138, -0.09487187, -0.10906488,  0.24568687, -0.32574353,
       -0.09812591,  0.38190797,  0.50065655, -0.6000787 ,  0.29480347,
       -0.1102195 ,  0.09189848, -0.10850241,  0.34490606, -0.34320882,
        0.22529666,  0.23808257,  0.22599994,  0.07547922,  0.34988502,
        0.02464161, -0.11065204,  0.7625838 , -0.56317943, -0.23231241,
        0.52497655, -0.39255306, -0.7996937 ,  0.11458381, -0.7375542 ,
       -0.23194315,  0.281631  , -0.59650177, -0.44351617, -0.53230256,
        0.15749614,  0.66059107,  0.2359882 ,  0.01045387, -0.1715315 ,
        0.1062969 , -0.36355722,  0.643036  , -0.24985766,  0.6084954 ,
        0.8096315 , -1.3040098 ,  0.05442179, -0.65849423, -0.01331906,
        0.21130672,  0.1566588 , -1.2179389 , -0.7748289 ,  0.16874771,
        0.29861358,  0.39398307, -0.18725336, -0.07219707,  0.31

In [76]:
toy_vector

array([ 0.00039017, -0.01238247,  0.03578262,  0.00090308,  0.00710632,
        0.00137028,  0.02872787, -0.00205029,  0.00709597,  0.01688072,
        0.01170911, -0.01519036, -0.01800711,  0.01046418,  0.00738228,
       -0.00738027,  0.02148806,  0.02002275, -0.02383891,  0.00746094,
        0.00354295,  0.02138529, -0.00567409,  0.0198663 ,  0.00997763,
        0.00987985,  0.04095101,  0.03317755,  0.01199262, -0.00126008,
        0.01511723,  0.01189682,  0.02140596, -0.03624062,  0.02774235,
        0.02886336,  0.00668471, -0.03119276,  0.00995038, -0.00113635,
       -0.00142319,  0.00567669, -0.0182303 , -0.00989127, -0.00527925,
        0.02816091,  0.0064056 ,  0.02119739, -0.01530181, -0.00073159,
        0.01653981,  0.00631407,  0.00346209, -0.0099992 , -0.02515699,
        0.01123337, -0.04449842,  0.02297432, -0.00128543,  0.02110685,
        0.02061981,  0.0009486 , -0.04445238, -0.00515058,  0.05133826,
        0.00973491,  0.01245662, -0.01554864,  0.00505119,  0.02

In [77]:
np.dot(yeon_vector, toy_vector)

0.50357443

In [78]:
my_fav = ["tarzan (1999)", "toy story (1995)", "cinderella (1950)",
         "beauty and the beast (1991)", "bambi (1942)"]

In [79]:
tarzan = title_to_idx["tarzan (1999)"]
tarzan_vector = als_model.item_factors[tarzan]
print(np.dot(yeon_vector, tarzan_vector))

cin = title_to_idx["cinderella (1950)"]
cin_vector = als_model.item_factors[cin]
print(np.dot(yeon_vector, cin_vector))

beauty = title_to_idx["beauty and the beast (1991)"]
beauty_vector = als_model.item_factors[beauty]
print(np.dot(yeon_vector, beauty_vector))

bambi = title_to_idx["bambi (1942)"]
bambi_vector = als_model.item_factors[bambi]
print(np.dot(yeon_vector, bambi_vector))

0.38375247
0.38752672
0.57635146
0.41738373


> 동일한 장르만 추가해서 1에 더 가까운 높은 값을 기대했는데 1st-try와 결과가 드라마틱하게 차이나지 않았다.
> - `title`과 `counts`로 콘텐츠 기반 필터링을 사용해 기대했던 것 만큼 `genre`와 깊은 관계가 있지 않다는 생각이 들었다.

In [80]:
# 보지 못한 영화에 대한 선호도
one = title_to_idx["one flew over the cuckoo's nest (1975)"] # Drama
one_vector = als_model.item_factors[one]
print(np.dot(yeon_vector, one_vector))

six = title_to_idx["sixth sense, the (1999)"] # Thriller
six_vector = als_model.item_factors[six]
print(np.dot(yeon_vector, six_vector))

-0.0184083
0.031199781


> 보지 못한 영화에 대한 선호도는 대부분 0 근처거나 음수임을 알 수 있다.
> - 1st-try보다는 차이가 많이 난다.

In [81]:
# 비슷한 영화 추천받기
fav_title = "toy story (1995)"
title_id = title_to_idx[fav_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(40, 0.9999999),
 (50, 0.7975026),
 (322, 0.6030621),
 (33, 0.5977157),
 (4, 0.57841027),
 (110, 0.52970064),
 (330, 0.48881882),
 (20, 0.4782641),
 (10, 0.47476017),
 (255, 0.42414254),
 (126, 0.39045945),
 (160, 0.37999249),
 (34, 0.37962848),
 (16, 0.36829722),
 (1825, 0.35697356)]

In [82]:
# index로부터 title 이름을 얻는 dict 생성
idx_to_title = {v: k for k, v in title_to_idx.items()}
# [idx_to_title[i[0]] for i in similar_title]

In [83]:
# 위를 함수로 구현하기
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

get_similar_title(fav_title) # "toy story (1995)"

['toy story (1995)',
 'toy story 2 (1999)',
 'babe (1995)',
 'aladdin (1992)',
 "bug's life, a (1998)",
 'groundhog day (1993)',
 'lion king, the (1994)',
 'pleasantville (1998)',
 'beauty and the beast (1991)',
 "there's something about mary (1998)"]

> 제목만 봐도 모두 `Animation` 영화임을 알 수 있다.

In [84]:
# 내가 좋아할만한 영화 추천
user = user_to_idx['yeon']

# recommend에선 user*item CSR Matrix를 받음
title_recommended = als_model.recommend(user,
                                        csr_data,
                                        N=20,
                                        filter_already_liked_items=True)
title_recommended

[(33, 0.51885164),
 (330, 0.47943527),
 (50, 0.47207084),
 (4, 0.39573058),
 (8, 0.37989622),
 (34, 0.36496577),
 (528, 0.3455613),
 (46, 0.33170196),
 (572, 0.32616752),
 (536, 0.3219366),
 (548, 0.31761724),
 (30, 0.3126245),
 (191, 0.3079758),
 (950, 0.2967661),
 (547, 0.2919436),
 (551, 0.28474638),
 (32, 0.2535414),
 (619, 0.24108833),
 (851, 0.23492028),
 (520, 0.23277721)]

In [85]:
[idx_to_title[i[0]] for i in title_recommended]

['aladdin (1992)',
 'lion king, the (1994)',
 'toy story 2 (1999)',
 "bug's life, a (1998)",
 'snow white and the seven dwarfs (1937)',
 'mulan (1998)',
 'pinocchio (1940)',
 'dumbo (1941)',
 'sleeping beauty (1959)',
 'jungle book, the (1967)',
 'fantasia (1940)',
 'antz (1998)',
 'little mermaid, the (1989)',
 '101 dalmatians (1961)',
 'lady and the tramp (1955)',
 'peter pan (1953)',
 'hercules (1997)',
 'alice in wonderland (1951)',
 'iron giant, the (1999)',
 "charlotte's web (1973)"]

In [86]:
aladdin = title_to_idx['aladdin (1992)']
explain = als_model.explain(user, csr_data, itemid=aladdin)

In [87]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('beauty and the beast (1991)', 0.22645449233092538),
 ('toy story (1995)', 0.14473115543218437),
 ('tarzan (1999)', 0.08401910869690082),
 ('cinderella (1950)', 0.034960320226042785),
 ('bambi (1942)', 0.021133182191141235)]

> 마찬가지로 내가 추가로 입력한 모든 영화들이 `aladdin`을 추천하는데 가장 많이 기여함을 알 수 있다.

***

# 🧤 회고

- 평소 추천시스템을 다루는 알고리즘에 대해 큰 관심이 있었는데 지난 Fundamental만 보고는 개념적인 이해도 완벽하게 되지 않았다. 이번 Exploration에서도 CSR Matrix나 MF 모델에 대해 100% 이해했다고 자부할 순 없지만 왜 사용하는지, 어떤 모델을 어떤 방식으로 사용할 수 있는지 조금은 배울 수 있었던 재밌는 노드였다.

<br/>

- 그동안 명시적인 데이터에 대해서만 생각해보았지 implicit한 피드백을 어떻게 처리할지 깊게 생각해보지 못했다. 이는 데이터를 다루는 사람의 주관과 데이터의 baseline을 모두 고려해야 하며 정답이 없다고 생각하는데, 다음 번엔 좀 더 밀접하고 맞물려있는 조건들도 분석해보고 싶다.

<br/>

- 처음엔 user_id와 movie_id가 1부터 차례대로 존재할 거라 생각해(원래 숫자로 주어졌기 때문에) 굳이 예제처럼 새로 영어 이름(ex. `yeon`)을 추가하고, 이를 `user_to_idx`로 변형하는 작업을 거쳐야 하나 싶었는데, 초기 nunique()로 파악해보니 중간에 빈 id 값들이 존재했다. EDA 때 이런 파악은 굉장히 중요한 부분이라 생각된다.

<br/>

- 진행하며 가장 궁금했던 부분은 어떤 원리로 1에 가까운게 선호도가 높은 것인지, 1보다 큰 것은 어떤 의미이며며 1보다 얼마나 가까워야 높은 선호도를 갖는다고 판단할 수 있는지였다 (..) 이것도 주관적인 견해가 들어가야 하는 것일까 ?!?! 아님 내가 모르는 것 ?!?!

<br/>

- Movielens 영화 추천 실습에서 genre는 explicit한 데이터로 콘텐츠 기반 필터링을 통해 추천해줄 수 있는 것 같다. `협업 필터링`과 `콘텐츠 기반 필터링`을 함께 사용할 순 없을까? 사실 counts와 genre를 동시에 고려해 사용자 맞춤형 영화를 추천해주고 싶었다. 분명 방법을 있을테지만 아직 거기까진 깊이 들어가지 못했다.

<br/>

- 1st-try에선 어떤 영화인지 고려하지 않고 아무 영화들을 추가해 담았었는데, 내가 본 영화에 대한 & 보지 못한 영화에 대한 선호도가 크게 차이가 나지 않았다. recommend로 내가 좋아할 만한 영화를 추천받았을 때도 그 영화가 정말 내가 추가한 영화들과 관련이 있는지도 잘 알지 못해 (허헛^__^..) 2nd-try를 진행했는데, 이 땐 `animation` 장르의 영화 위주로 추가했다. 1st-try보단 계산된 선호도가 조금 높긴 했지만 별반 다를 건 없었다. 3번째로 작성한 선호도 측정 값에 대한 이해가 되어야 해결할 수 있는 문제인 것 같다.

<br/>

- 또한 기여도 체크 부분에서 내가 새롭게 추가한 영화들이 새 영화들을 추천하는데 가장 많이 기여한 것을 보고 어쩌면 당연한 건데 원래 이런 건가..? 싶은 의문도 들었다.

<br/>

- 평소 알고리즘의 노예로 사는 것을 좋아해서 (..) 여러 유투브, 넷플릭스 등의 OTT 서비스들 혹은 특히 멜론에서 내 선호도를 파악해 추천해주는 음악이나 한 해가 끝나고 1년 간의 내 데이터를 분석해 리포트로 보여주는 것을 굉장히 좋아한다. 나도 추천 알고리즘의 개발자로 실생활에 유의미한 도움을 줄 수 있는 삶을 살고싶어졌다. 재밌었던 노드다!